In [ ]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import numpy as np


(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Creating a list of all the class labels
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Converting the pixels data to float type
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

# Normalize input
train_images = train_images / 255
test_images = test_images / 255

# change target class to one hot encoding
# use ‘to_categorical’ function

num_classes = 10
Y_train = to_categorical(train_labels, num_classes)
Y_test = to_categorical(test_labels, num_classes)

# Check the resulting shapes
print("Y_train shape after to_categorical:", Y_train.shape)
print("Y_test shape after to_categorical:", Y_test.shape)


# Change target class to one hot encoding using 'to_categorical' function
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)


Y_train shape after to_categorical: (50000, 10)
Y_test shape after to_categorical: (10000, 10)


In [ ]:
X_train=train_images
X_test=test_images


# Show the shape of the datasets

print("Shape of train_images:", X_train.shape)
print("Shape of train_labels:", Y_train.shape)
print("Shape of test_images:", X_test.shape)
print("Shape of test_labels:", Y_test.shape)


Shape of train_images: (50000, 32, 32, 3)
Shape of train_labels: (50000, 10)
Shape of test_images: (10000, 32, 32, 3)
Shape of test_labels: (10000, 10)


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

def create_model(input_shape, fine_tune=0):
    # Load the VGG16 model with pretrained weights, excluding the top layers
    conv_base = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze layers according to the fine_tune parameter
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Build the model with resizing, convolutional base, and fully connected layers
    model = models.Sequential([
        layers.Resizing(224, 224, interpolation="bilinear", input_shape=input_shape),  # Resizing layer
        conv_base,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),  # First dense layer with 128 nodes
        layers.Dense(64, activation='relu'),   # Second dense layer with 64 nodes
        layers.Dense(10, activation='softmax')  # Output layer for 10 classes
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



In [ ]:
input_shape = X_train[0].shape
model = create_model(input_shape, fine_tune=0)
print(model.summary())

# Train the model
history = model.fit(X_train, Y_train,
                    epochs=1,
                    batch_size=64,
                    validation_data=(X_test, Y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=2)

# Print test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot accuracy and loss
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resizing (Resizing)                  │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,934,986 (68.42 MB)

 Trainable params: 3,220,298 (12.28 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

None
 12/782 ━━━━━━━━━━━━━━━━━━━━ 9:12:14 43s/step - accuracy: 0.1002 - loss: 2.6975

In [ ]:
def train_with_fine_tune_layers(num_trainable_layers, input_shape):
    # Create the model with a specific number of layers trainable
    model = create_model(input_shape, fine_tune=num_trainable_layers)

    # Train the model
    history = model.fit(
        X_train, Y_train,
        epochs=10,  # You can adjust the epochs based on training time
        batch_size=64,
        validation_data=(X_test, Y_test),
        verbose=0  # Set to 0 to suppress detailed output
    )

    # Get the final validation accuracy for the model
    final_val_accuracy = history.history['val_accuracy'][-1]
    return final_val_accuracy

# Define input shape and number of layers to try
input_shape = X_train[0].shape
trainable_layers_range = range(0, len(VGG16(include_top=False).layers) + 1, 5)  # Adjust step size as needed
accuracies = []

for layers in trainable_layers_range:
    print(f"Training with {layers} trainable layers...")
    accuracy = train_with_fine_tune_layers(layers, input_shape)
    accuracies.append(accuracy)
    print(f"Validation Accuracy with {layers} trainable layers: {accuracy:.4f}")

plt.plot(trainable_layers_range, accuracies, marker='o')
plt.xlabel('Number of Trainable Layers')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy vs. Number of Trainable Layers')
plt.grid(True)
plt.show()

# Find the optimal number of layers
optimal_layers = trainable_layers_range[accuracies.index(max(accuracies))]
print(f"Optimal number of trainable layers: {optimal_layers}")


In [ ]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Reshape to (num_samples, 28, 28, 1) and then resize to (224, 224, 3)
train_images = np.expand_dims(train_images, axis=-1)  # Add channel dimension (grayscale)
test_images = np.expand_dims(test_images, axis=-1)

# Resize the images to 224x224 and convert grayscale to 3 channels
train_images_resized = np.array([tf.image.resize(img, (224, 224)) for img in train_images])
test_images_resized = np.array([tf.image.resize(img, (224, 224)) for img in test_images])

# Convert grayscale to 3 channels (RGB)
train_images_resized = np.repeat(train_images_resized, 3, axis=-1)
test_images_resized = np.repeat(test_images_resized, 3, axis=-1)

# Normalize the images
train_images_resized = train_images_resized.astype('float32') / 255
test_images_resized = test_images_resized.astype('float32') / 255

# One-hot encode the labels
num_classes = 10
train_labels_one_hot = to_categorical(train_labels, num_classes)
test_labels_one_hot = to_categorical(test_labels, num_classes)

# Check the shapes
print("Shape of train_images_resized:", train_images_resized.shape)
print("Shape of test_images_resized:", test_images_resized.shape)
print("Shape of train_labels_one_hot:", train_labels_one_hot.shape)
print("Shape of test_labels_one_hot:", test_labels_one_hot.shape)

# Use the same function for training the model with varying trainable layers
def train_with_fine_tune_layers_mnist(num_trainable_layers, input_shape):
    # Create the model with a specific number of layers trainable
    model = create_model(input_shape, fine_tune=num_trainable_layers)

    # Train the model
    history = model.fit(
        train_images_resized, train_labels_one_hot,
        epochs=1,  # You can adjust the epochs based on training time
        batch_size=64,
        validation_data=(test_images_resized, test_labels_one_hot),
        verbose=0  # Set to 0 to suppress detailed output
    )

    # Get the final validation accuracy for the model
    final_val_accuracy = history.history['val_accuracy'][-1]
    return final_val_accuracy

# Define input shape for MNIST
input_shape_mnist = train_images_resized[0].shape
trainable_layers_range = range(0, len(VGG16(include_top=False).layers) + 1, 5)  # Adjust step size as needed
accuracies_mnist = []

# Train with varying number of trainable layers
for layers in trainable_layers_range:
    print(f"Training with {layers} trainable layers...")
    accuracy = train_with_fine_tune_layers_mnist(layers, input_shape_mnist)
    accuracies_mnist.append(accuracy)
    print(f"Validation Accuracy with {layers} trainable layers: {accuracy:.4f}")


import matplotlib.pyplot as plt

plt.plot(trainable_layers_range, accuracies_mnist, marker='o')
plt.xlabel('Number of Trainable Layers')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy vs. Number of Trainable Layers (MNIST)')
plt.grid(True)
plt.show()

# Find the optimal number of layers
optimal_layers_mnist = trainable_layers_range[accuracies_mnist.index(max(accuracies_mnist))]
print(f"Optimal number of trainable layers for MNIST: {optimal_layers_mnist}")
